# Построение и оптимизация модели

В этом задании вам предстоит поучаствовать в соревновании на [kaggle inclass](https://inclass.kaggle.com/c/telecom-clients-churn-prediction).
В соревновании вы будете работать с той же выборкой, что и ранее, поэтому воспользуйтесь результатами, полученными на предыдущих неделях. Для успешного участия в соревновании необходимо преодолеть по качеству beseline решение.

Итак, мы научились обрабатывать данные, выбрали схему кросс-валидации и определились с метриками качества. Пора переходить к оптимизации модели. На этой неделе вам предстоит принять участие в соревновании на платформе kaggle inclass! Цель такого соревнования - преодолеть предложенное baseline решение, а, главное, обсудить и сравнить предложенные решения на форуме. Какие признаки оказали наибольший вклад в модель? Как лучше обрабатывать категориальные признаки? Нужно ли делать отбор признаков, А балансировать выборку? Экспериментируйте с данными и обсуждайте ваши решения на форуме!

In [1]:
import pandas as pd
import random
import numpy as np
from scipy.sparse import coo_matrix, hstack
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import roc_curve, precision_recall_curve, f1_score, roc_auc_score, recall_score, precision_score, log_loss

from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

seed = 1903
first_categorial_index = 190

Загрузим train dataset.

In [2]:
churn_data_frame = pd.read_csv("..\..\Data\churn_data_train.csv", ",")
churn_labels_frame = pd.read_csv("..\..\Data\churn_labels_train.csv")
print(churn_data_frame.shape)
print(churn_labels_frame.shape)

(27999, 230)
(27999, 1)


## Общая предобработка признаков

In [19]:
def scale_frame(frame):
    """Функция масштабирает frame на отрезок [0;1]"""
    scaler = StandardScaler()
    scaled_matrix = scaler.fit_transform(frame.as_matrix())
    return pd.DataFrame(scaled_matrix, columns=frame.columns)

def fill_numericna_means(frame):
    """ Функция заполняет значения в числовом фрейме средними и удаляет те колонке, в которых значений нет. """
    n_frame = frame
    
    # Посчитаем средние по колонкам
    numeric_means = n_frame.mean(axis=0, skipna=True)
    # Оставим только те колонки, в которых среднее значение не равно NaN, т.к. в таких колонках совсем нет значений
    numeric_means = numeric_means.dropna()
    dropped_columns = n_frame.columns.drop(numeric_means.index)
    n_frame = n_frame[list(numeric_means.index)]
    # Заполним пропущенные численные значения средними
    n_frame = n_frame.fillna(numeric_means, axis=0)
    return (n_frame, dropped_columns)
    
def remove_constant_features(frame):
    """Функция удаляет колонки, которые содержат только одно значение."""
    
    # Посчитаем количества уникальных значений по колонкам
    unique_counts = frame.nunique()
    # Удалим колонки с одним уникальным значением
    columns_to_drop = unique_counts[unique_counts <= 1].index
    
    return (frame.drop(columns=columns_to_drop), columns_to_drop)

Разделим коллекции на группы - числовые и категориальные.

In [20]:
numeric_columns = churn_data_frame.columns[:first_categorial_index]
categorial_columns = churn_data_frame.columns[first_categorial_index:]

numeric_frame = churn_data_frame[numeric_columns].copy()
categorial_frame = churn_data_frame[categorial_columns].copy()

Удалим вещественные колонки, содержащие одно и менее значений

In [28]:
numeric_frame_no_const, dropped_const_numeric_columns = remove_constant_features(numeric_frame)

Заполним пропущенные вещественные значения средними

In [29]:
numeric_frame_means, dropped_na_numeric_columns = fill_numericna_means(numeric_frame_no_const)

Заполним пропущенные категориальные значения строками "NaV" (Not a value)

In [30]:
categorial_frame_nav = categorial_frame.fillna("NaV")

Удалим категориальные колонки с одним единственным значением

In [31]:
categorial_frame_nav_no_const, dropped_categorial_columns = remove_constant_features(categorial_frame_nav)

Список удаленных колонок

In [34]:
dropped_columns = np.concatenate([
        list(dropped_const_numeric_columns),
        list(dropped_na_numeric_columns),
        list(dropped_categorial_columns)])
print (", ".join(dropped_columns))

Var8, Var15, Var20, Var31, Var32, Var39, Var42, Var48, Var52, Var55, Var79, Var118, Var141, Var167, Var169, Var175, Var185, Var209, Var230


In [ ]:
# Заполнить пропущенные вещественные значения медианами
# Заполнить пропущенные категориальные значения самой частой категорией
# Масштабировать
# Oversampling/Undersampling/No sampling
# Удалить признаки не коррелирующие с целевой переменной или добавлять по одному признаку по убыванию корреляции,
#   пока растет качество.
# Удалить признаки сильно коррелирующие друг с другом или удалять по одному, пока растет качество.
# Автоматический отсев признаков

# Для линейной модели попробовать бинаризацию вместо OneHot
# Для остальных попробовать бинаризацию вместо Labeled
# Попробовать другие способы работы с категориальными признаками

# Поискать скоррелированные значения между категориальными и численными признаками